In [1]:
!pip install summac

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 2.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of transformers to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 43.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 37.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 50.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 48.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of transformers to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 42.0 MB/s 

In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [8]:
from summac.model_summac import SummaCZS, SummaCConv

model_zs = SummaCZS(granularity="sentence", model_name="vitc", device="cpu") # If you have a GPU: switch to: device="cuda"
model_conv = SummaCConv(models=["vitc"], bins='percentile', granularity="sentence", nli_labels="e", device="cpu", start_file="default", agg="mean")

document = """Scientists are studying Mars to learn about the Red Planet and find landing sites for future missions.
One possible site, known as Arcadia Planitia, is covered instrange sinuous features.
The shapes could be signs that the area is actually made of glaciers, which are large masses of slow-moving ice.
Arcadia Planitia is in Mars' northern lowlands."""

summary1 = "There are strange shape patterns on Arcadia Planitia. The shapes could indicate the area might be made of glaciers. This makes Arcadia Planitia ideal for future missions."
score_zs1 = model_zs.score([document], [summary1])
score_conv1 = model_conv.score([document], [summary1])
print("[Summary 1] SummaCZS Score: %.3f; SummacConv score: %.3f" % (score_zs1["scores"][0], score_conv1["scores"][0])) # [Summary 1] SummaCZS Score: 0.582; SummacConv score: 0.536

summary2 = "There are strange shape patterns on Arcadia Planitia. The shapes could indicate the area might be made of glaciers."
score_zs2 = model_zs.score([document], [summary2])
score_conv2 = model_conv.score([document], [summary2])
print("[Summary 2] SummaCZS Score: %.3f; SummacConv score: %.3f" % (score_zs2["scores"][0], score_conv2["scores"][0])) # [Summary 2] SummaCZS Score: 0.877; SummacConv score: 0.709

<All keys matched successfully>
[Summary 1] SummaCZS Score: 0.582; SummacConv score: 0.536
[Summary 2] SummaCZS Score: 0.877; SummacConv score: 0.709


In [ ]:
from tqdm import tqdm
import pandas as pd
from summac.model_summac import SummaCZS, SummaCConv

# Load the CSV file
df = pd.read_csv("Bangla1.csv")

# Extract the predictions and references columns
predictions = df["New"].tolist()
references = df["question"].tolist()

# Initialize the SummaC evaluators
model_zs = SummaCZS(granularity="sentence", model_name="vitc", device="cpu")  # If you have a GPU: switch to: device="cuda"
model_conv = SummaCConv(models=["vitc"], bins='percentile', granularity="sentence", nli_labels="e", device="cpu", start_file="default", agg="mean")

# Compute the SummaC scores for each prediction-reference pair
scores_zs = []
scores_conv = []
n_predictions = len(predictions)
with tqdm(total=n_predictions) as pbar:
    for i in range(n_predictions):
        # Compute the SummaC scores for the current prediction-reference pair
        score_zs = model_zs.score([predictions[i]], [references[i]])
        score_conv = model_conv.score([predictions[i]], [references[i]])

        # Print the immediate scores
        print(f"SummaCZS Score for prediction {i+1}: {score_zs['scores'][0]:.3f}")
        print(f"SummaCConv Score for prediction {i+1}: {score_conv['scores'][0]:.3f}")

        # Append the scores to the list
        scores_zs.append(score_zs["scores"][0])
        scores_conv.append(score_conv["scores"][0])

        # Update the progress bar
        pbar.update(1)
        pbar.set_description("Processing predictions... (%d/%d)" % (i+1, n_predictions))

# Compute the average scores
avg_score_zs = sum(scores_zs) / len(scores_zs)
avg_score_conv = sum(scores_conv) / len(scores_conv)

# Print the average scores
print("Average SummaCZS Score: %.3f; Average SummacConv score: %.3f" % (avg_score_zs, avg_score_conv))


<All keys matched successfully>


  0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/summac/model_summac.py:298: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  histograms = torch.FloatTensor(histograms).to(self.device)
Processing predictions... (1/3):  33%|███▎      | 1/3 [00:24<00:48, 24.16s/it]

SummaCZS Score for prediction 1: 0.347
SummaCConv Score for prediction 1: 0.222


Processing predictions... (2/3):  67%|██████▋   | 2/3 [00:48<00:23, 23.99s/it]

SummaCZS Score for prediction 2: 0.507
SummaCConv Score for prediction 2: 0.218


Processing predictions... (3/3): 100%|██████████| 3/3 [01:11<00:00, 23.86s/it]

SummaCZS Score for prediction 3: 0.230
SummaCConv Score for prediction 3: 0.214
Average SummaCZS Score: 0.361; Average SummacConv score: 0.218


# Finetuned summary

In [13]:
from tqdm import tqdm
import pandas as pd
from summac.model_summac import SummaCZS, SummaCConv

# Load the CSV file
df = pd.read_csv("finetuned-summaries-1.csv")

# Extract the predictions and references columns
predictions = df["Predicted_Summary"].tolist()
references = df["Reference_Text"].tolist()

# Initialize the SummaC evaluators
model_zs = SummaCZS(granularity="document", model_name="vitc", device="cuda")  # If you have a GPU: switch to: device="cuda"
model_conv = SummaCConv(models=["vitc"], bins='percentile', granularity="document", nli_labels="e", device="cuda", start_file="default", agg="mean")

# Compute the SummaC scores for each prediction-reference pair
scores_zs = []
scores_conv = []
n_predictions = len(predictions)
with tqdm(total=n_predictions) as pbar:
    for i in range(n_predictions):
        # Compute the SummaC scores for the current prediction-reference pair
        score_zs = model_zs.score( [references[i]], [predictions[i]])
        score_conv = model_conv.score([references[i]], [predictions[i]])

        # Print the immediate scores
        print(f"SummaCZS Score for prediction {i+1}: {score_zs['scores'][0]:.5f}")
        print(f"SummaCConv Score for prediction {i+1}: {score_conv['scores'][0]:.5f}")

        # Append the scores to the list
        scores_zs.append(score_zs["scores"][0])
        scores_conv.append(score_conv["scores"][0])

        # Update the progress bar
        pbar.update(1)
        pbar.set_description("Processing predictions... (%d/%d)" % (i+1, n_predictions))

# Compute the average scores
avg_score_zs = sum(scores_zs) / len(scores_zs)
avg_score_conv = sum(scores_conv) / len(scores_conv)

# Print the average scores
print("Average SummaCZS Score: %.5f; Average SummacConv score: %.5f" % (avg_score_zs, avg_score_conv))


<All keys matched successfully>


Processing predictions... (2/200):   1%|          | 2/200 [00:03<04:57,  1.50s/it]

SummaCZS Score for prediction 1: 0.22496
SummaCConv Score for prediction 1: 0.21373
SummaCZS Score for prediction 2: 0.95194
SummaCConv Score for prediction 2: 0.46412


Processing predictions... (6/200):   3%|▎         | 6/200 [00:03<01:23,  2.33it/s]

SummaCZS Score for prediction 3: 0.15512
SummaCConv Score for prediction 3: 0.21373
SummaCZS Score for prediction 4: 0.43821
SummaCConv Score for prediction 4: 0.22153
SummaCZS Score for prediction 5: -0.00125
SummaCConv Score for prediction 5: 0.19352
SummaCZS Score for prediction 6: 0.39056
SummaCConv Score for prediction 6: 0.22153


...

SummaCZS Score for prediction 192: -0.68756
SummaCConv Score for prediction 192: 0.20179
SummaCZS Score for prediction 193: 0.93267
SummaCConv Score for prediction 193: 0.35291
SummaCZS Score for prediction 194: 0.47754
SummaCConv Score for prediction 194: 0.22153
SummaCZS Score for prediction 195: 0.12064
SummaCConv Score for prediction 195: 0.21373


Processing predictions... (198/200):  99%|█████████▉| 198/200 [00:14<00:00, 16.49it/s]

SummaCZS Score for prediction 196: 0.00873
SummaCConv Score for prediction 196: 0.19352
SummaCZS Score for prediction 197: 0.05626
SummaCConv Score for prediction 197: 0.20179
SummaCZS Score for prediction 198: 0.00239
SummaCConv Score for prediction 198: 0.20011


Processing predictions... (200/200): 100%|██████████| 200/200 [00:14<00:00, 13.90it/s]

SummaCZS Score for prediction 199: 0.08325
SummaCConv Score for prediction 199: 0.21373
SummaCZS Score for prediction 200: 0.96443
SummaCConv Score for prediction 200: 0.55624
Average SummaCZS Score: 0.30544; Average SummacConv score: 0.23866


# Without finetuning

In [15]:
from tqdm import tqdm
import pandas as pd
from summac.model_summac import SummaCZS, SummaCConv

# Load the CSV file
df = pd.read_csv("zero-shot-summaries-1.csv")

# Extract the predictions and references columns
predictions = df["ExtractedSummary"].tolist()
references = df["CHQ"].tolist()

# Initialize the SummaC evaluators
model_zs = SummaCZS(granularity="sentence", model_name="vitc", device="cuda")  # If you have a GPU: switch to: device="cuda"
model_conv = SummaCConv(models=["vitc"], bins='percentile', granularity="document", nli_labels="e", device="cuda", start_file="default", agg="mean")

# Compute the SummaC scores for each prediction-reference pair
scores_zs = []
scores_conv = []
n_predictions = len(predictions)
with tqdm(total=n_predictions) as pbar:
    for i in range(n_predictions):
        # Compute the SummaC scores for the current prediction-reference pair
        score_zs = model_zs.score( [references[i]], [predictions[i]])
        score_conv = model_conv.score([references[i]], [predictions[i]])

        # Print the immediate scores
        print(f"SummaCZS Score for prediction {i+1}: {score_zs['scores'][0]:.5f}")
        print(f"SummaCConv Score for prediction {i+1}: {score_conv['scores'][0]:.5f}")

        # Append the scores to the list
        scores_zs.append(score_zs["scores"][0])
        scores_conv.append(score_conv["scores"][0])

        # Update the progress bar
        pbar.update(1)
        pbar.set_description("Processing predictions... (%d/%d)" % (i+1, n_predictions))

# Compute the average scores
avg_score_zs = sum(scores_zs) / len(scores_zs)
avg_score_conv = sum(scores_conv) / len(scores_conv)

# Print the average scores
print("Average SummaCZS Score: %.5f; Average SummacConv score: %.5f" % (avg_score_zs, avg_score_conv))


<All keys matched successfully>


Processing predictions... (3/343):   1%|          | 3/343 [00:03<04:55,  1.15it/s]

SummaCZS Score for prediction 1: 0.72388
SummaCConv Score for prediction 1: 0.55624
SummaCZS Score for prediction 2: 0.67346
SummaCConv Score for prediction 2: 0.22153
SummaCZS Score for prediction 3: 0.96925
SummaCConv Score for prediction 3: 0.46412


Processing predictions... (6/343):   2%|▏         | 6/343 [00:03<02:36,  2.16it/s]

SummaCZS Score for prediction 4: -0.01434
SummaCConv Score for prediction 4: 0.21373
SummaCZS Score for prediction 5: 0.96162
SummaCConv Score for prediction 5: 0.76738
SummaCZS Score for prediction 6: -0.32751
SummaCConv Score for prediction 6: 0.22153


...

SummaCZS Score for prediction 337: -0.02375
SummaCConv Score for prediction 337: 0.86952
SummaCZS Score for prediction 338: 0.48438
SummaCConv Score for prediction 338: 0.76738
SummaCZS Score for prediction 339: 0.43598
SummaCConv Score for prediction 339: 0.22153


Processing predictions... (342/343): 100%|█████████▉| 342/343 [00:36<00:00,  8.68it/s]

SummaCZS Score for prediction 340: -0.48254
SummaCConv Score for prediction 340: 0.22153
SummaCZS Score for prediction 341: 0.95959
SummaCConv Score for prediction 341: 0.76738
SummaCZS Score for prediction 342: -0.40774
SummaCConv Score for prediction 342: 0.22153


Processing predictions... (343/343): 100%|██████████| 343/343 [00:36<00:00,  9.27it/s]

SummaCZS Score for prediction 343: -0.00743
SummaCConv Score for prediction 343: 0.36450
Average SummaCZS Score: 0.28194; Average SummacConv score: 0.46748
